In [1]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('english'))


In [2]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(str.maketrans('', '', string.punctuation))
    filter_str = final_str.translate(str.maketrans('', '', digits))
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [3]:
df_test = pd.read_csv('test_dataset.csv')
df_test.head()

,ID,Essayset,min_score,max_score,clarity,coherent,EssayText
0,1673,1,0,3,average,worst,The procedures I think they should have includ...
1,1674,1,0,3,average,worst,"In order to replicate this experiment, you wou..."
2,1675,1,0,3,above_average,above_average,"In order to replicate their experiment, you wo..."
3,1676,1,0,3,worst,worst,Pleace a simple of one material into one conta...
4,1677,1,0,3,worst,worst,Determin the mass of four different samples ma...


In [4]:
df_train = pd.read_csv('train_dataset.csv')

In [5]:
df_train['source_'] = 1
df_test['source_'] =0

In [6]:
df = pd.concat([df_train,df_test])
df.loc[df['source_']==0]
df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
multi_datasets = {}
groups = df.groupby('Essayset')
for name, group in groups:
    multi_datasets[name] = group

In [8]:
single_df = multi_datasets[2]
print(single_df.shape)
single_df.head()
single_df.reset_index(inplace= True)


(1696, 13)


In [9]:
single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)


ValueError: cannot convert float NaN to integer

In [430]:
print(single_df.shape)
single_df.head()


(1225, 8)


,EssayText,Essayset,clarity,coherent,max_score,min_score,source_,candi_score
0,Changing the type of grafin would improve the ...,2.0,average,worst,3,0,1,1
1,Concluding from the students data that plastic...,2.0,excellent,above_average,3,0,1,3
2,Two ways that the stundent could've improved t...,2.0,average,worst,3,0,1,1
3,A conclusion I can make from this experiment i...,2.0,average,worst,3,0,1,1
4,a.One conclusion I can draw is that plastic B ...,2.0,above_average,average,3,0,1,3


In [431]:
#New feture percentage match 


In [432]:
single_df['kitna_aacha']=single_df['candi_score']

In [433]:
print(single_df.shape)
single_df.head()


(1225, 9)


,EssayText,Essayset,clarity,coherent,max_score,min_score,source_,candi_score,kitna_aacha
0,Changing the type of grafin would improve the ...,2.0,average,worst,3,0,1,1,1
1,Concluding from the students data that plastic...,2.0,excellent,above_average,3,0,1,3,3
2,Two ways that the stundent could've improved t...,2.0,average,worst,3,0,1,1,1
3,A conclusion I can make from this experiment i...,2.0,average,worst,3,0,1,1,1
4,a.One conclusion I can draw is that plastic B ...,2.0,above_average,average,3,0,1,3,3


In [435]:
single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)

In [436]:
print(single_df.shape)
single_df.head()

(1225, 5)


,EssayText,clarity,coherent,source_,kitna_aacha
0,Changing the type of grafin would improve the ...,average,worst,1,1
1,Concluding from the students data that plastic...,excellent,above_average,1,3
2,Two ways that the stundent could've improved t...,average,worst,1,1
3,A conclusion I can make from this experiment i...,average,worst,1,1
4,a.One conclusion I can draw is that plastic B ...,above_average,average,1,3


In [437]:
single_df['clarity'].value_counts()

excellent        412
above_average    365
average          235
worst            213
Name: clarity, dtype: int64

In [438]:
all_text  = []
single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
all_text = list(single_df['EssayText'])

In [400]:
len(all_text)

1225

In [439]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf.fit(all_text)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='latin-1', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=20,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [402]:
features = tfidf.transform(single_df['EssayText'])

In [403]:
single_df.head()

,level_0,EssayText,clarity,coherent,source_,kitna_aacha
0,1672,changing type grafin would improve student exp...,average,worst,1,1
1,1673,concluding student data plastic type b stretch...,excellent,above_average,1,3
2,1674,two way stundent couldve improved experiment g...,average,worst,1,1
3,1675,conclusion make experiment plastic type b stre...,average,worst,1,1
4,1676,aone conclusion draw plastic b stretched plast...,above_average,average,1,3


In [448]:
new_tfidf_features = pd.DataFrame(features.toarray())
new_tfidf_features['clarity'] = single_df['clarity']
new_tfidf_features['coherent'] = single_df['coherent']
new_tfidf_features['kitna_aacha'] = single_df['kitna_aacha']
new_tfidf_features['source_'] = single_df['source_']

print(new_tfidf_features.shape)
new_tfidf_features.head()

(1225, 290)


,0,1,2,3,4,5,6,7,8,9,...,280,281,282,283,284,285,clarity,coherent,kitna_aacha,source_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,excellent,above_average,3.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,above_average,average,3.0,1.0


In [449]:
new_tfidf_features.head()

,0,1,2,3,4,5,6,7,8,9,...,280,281,282,283,284,285,clarity,coherent,kitna_aacha,source_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,excellent,above_average,3.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,above_average,average,3.0,1.0


In [450]:
new_tfidf_features['clarity']=new_tfidf_features['clarity'].fillna(new_tfidf_features['clarity'].mode()[0])
new_tfidf_features['coherent']=new_tfidf_features['coherent'].fillna(new_tfidf_features['coherent'].mode()[0])


In [451]:
new_tfidf_features.head()

,0,1,2,3,4,5,6,7,8,9,...,280,281,282,283,284,285,clarity,coherent,kitna_aacha,source_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,excellent,above_average,3.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,average,worst,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,above_average,average,3.0,1.0


In [452]:
new_tfidf_features.dropna(axis=0,inplace=True)

In [453]:
new_tfidf_features.shape

(1181, 290)

In [454]:
all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity' ,'coherent'])

In [455]:
all_preprocessed_single_q_data.head()

,0,1,2,3,4,5,6,7,8,9,...,kitna_aacha,source_,clarity_above_average,clarity_average,clarity_excellent,clarity_worst,coherent_above_average,coherent_average,coherent_excellent,coherent_worst
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0,1,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0,0,1,0,1,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0,1,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0,1,0,0,0,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1,0,0,0,0,1,0,0


In [456]:
preprocessed_main_test = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 0]
preprocessed_main_train = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 1]

In [457]:
y_train = (preprocessed_main_train['kitna_aacha'])
x_train = preprocessed_main_train.drop(['kitna_aacha'],axis =1)

In [462]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =0)
X_train.shape

(1181, 295)

In [463]:
preprocessed_main_test

,0,1,2,3,4,5,6,7,8,9,...,kitna_aacha,source_,clarity_above_average,clarity_average,clarity_excellent,clarity_worst,coherent_above_average,coherent_average,coherent_excellent,coherent_worst


In [461]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = MultinomialNB()

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(preprocessed_main_test)



ValueError: Found array with 0 sample(s) (shape=(0, 296)) while a minimum of 1 is required.

In [362]:
from sklearn import metrics
metrics.mean_squared_error(y_test_internal, pred_internal)

ValueError: Found input variables with inconsistent numbers of samples: [0, 307]

In [373]:
pp = []
models_ = {}
def different_models(df):
    groups = df.groupby('Essayset')
    for name, group in groups:
        multi_datasets[name] = group
    pp = []
    models_ = {}
    for k in multi_datasets:
        print(k)
        single_df = multi_datasets[k]
        single_df.reset_index(inplace=True)
        single_df.dropna(axis=0,inplace=True)
        
        single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
        single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
        single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)

        single_df['kitna_aacha']=single_df['candi_score']
        
        single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)
        
        all_text  = []
        single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
        all_text = list(single_df['EssayText'])
        
        from sklearn.feature_extraction.text import TfidfVectorizer
        tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
        tfidf.fit(all_text)
        
        features = tfidf.transform(single_df['EssayText'])
        
        new_tfidf_features = pd.DataFrame(features.toarray())
        new_tfidf_features['clarity'] = single_df['clarity']
        new_tfidf_features['coherent'] = single_df['coherent']
        new_tfidf_features['kitna_aacha'] = single_df['kitna_aacha']
        
        new_tfidf_features['clarity']=new_tfidf_features['clarity'].fillna(new_tfidf_features['clarity'].mode()[0])
        new_tfidf_features['coherent']=new_tfidf_features['coherent'].fillna(new_tfidf_features['coherent'].mode()[0])

        new_tfidf_features.dropna(axis=0,inplace=True)
        
        all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity' ,'coherent'])
        
        y_train = (all_preprocessed_single_q_data['kitna_aacha'])
        x_train = all_preprocessed_single_q_data.drop(['kitna_aacha'],axis =1)
        
        from sklearn.model_selection import train_test_split
        from sklearn.naive_bayes import MultinomialNB,GaussianNB
        from sklearn.linear_model import LogisticRegression
        X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.1)
        
        # Import the model we are using
        from sklearn.ensemble import RandomForestClassifier

        # Instantiate model with 1000 decision trees
        rf = RandomForestClassifier()

        # Train the model on training data
        rm = rf.fit(X_train, y_train_internal)
        pred_internal = rm.predict(X_test)
        
        from sklearn import metrics
        pp.append(metrics.accuracy_score(y_test_internal, pred_internal))
        models_[k] = rf
        
    return ((pp,models_))

    

In [374]:
multi_datasets = {}

a,b = different_models(df)

1.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


3.0


...

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [375]:
np.array(a).mean()

0.6824935480419513

In [54]:
from bert_embedding import BertEmbedding

bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
 Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
 As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
BERT is conceptually simple and empirically powerful. 
It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""


In [58]:
sentences = bert_abstract.split('\n')
bert_embedding = BertEmbedding()
result = bert_embedding(sentences,'sum')

In [68]:
sentences

['We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.',
 ' Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.',
 ' As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. ',
 'BERT is conceptually simple and empirically powerful. ',
 'It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performanc

In [62]:
(result[1][0])
sen_vec = []
for single_sen,all_word_vec in result:
    comlete_sen_vector = np.array(all_word_vec).mean(axis =0)
    sen_vec.append((single_sen,comlete_sen_vector))

In [67]:
len(sen_vec[1][1])

768

In [34]:
import numpy as np
comlete_sen_vector = np.array(result[0][1]).mean(axis =0)
len(comlete_sen_vector)

768

In [28]:
np.array(result[0][1][1])

array([-7.83167362e-01,  5.98037302e-01,  3.33789326e-02, -6.20065510e-01,
        1.85064733e-01, -5.74877203e-01,  2.52220005e-01, -1.95262954e-02,
        3.05282533e-01, -3.45093995e-01, -6.90376401e-01,  4.64573614e-02,
       -5.02824366e-01,  3.89240310e-02, -8.15680474e-02,  5.61777830e-01,
       -3.94920826e-01, -4.24305767e-01,  7.19363391e-02,  4.78396654e-01,
        1.22789077e-01,  4.31831539e-01, -7.77603149e-01,  3.69261771e-01,
        6.10460699e-01, -1.27367902e+00,  1.01201904e+00, -1.95420161e-02,
       -1.89690247e-01,  1.87784255e-01, -2.26875678e-01,  8.03991854e-02,
        1.86895803e-02, -7.84316361e-02, -4.00059730e-01,  7.59272039e-01,
        5.80961823e-01, -1.64119095e-01, -4.28889811e-01,  4.24854219e-01,
       -1.17986172e-01, -4.35561955e-01,  6.06887698e-01,  5.70732296e-01,
        1.35500774e-01, -1.03446770e+00,  4.34642613e-01,  7.36153424e-02,
        1.23390570e-01, -1.01948641e-01, -1.02304471e+00,  2.02407390e-01,
        2.50787467e-01,  

In [107]:
def get_vector(bert_embedding,sentences):
    #sentences 
    result = bert_embedding(sentences,'sum')
    sen_vec = []
    for single_sen,all_word_vec in result:
        comlete_sen_vector = np.array(all_word_vec).mean(axis =0)
        sen_vec.append(comlete_sen_vector)
         
            
    return sen_vec

In [108]:
pp = get_vector(bert_embedding ,  ['a',''])

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice.
  
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [103]:
df =  pd.read_csv('train_dataset.csv')
print(df.head())
df = df.loc[df['Essayset'] ==1]
print(df.shape)
s = list(df['EssayText'])
len(s)

   ID  Essayset  min_score  max_score  score_1  score_2  score_3  score_4  \
0   1       1.0          0          3        1        1      1.0      1.0   
1   2       1.0          0          3        1        1      NaN      1.5   
2   3       1.0          0          3        1        1      1.0      1.0   
3   4       1.0          0          3        0        0      0.0      0.0   
4   5       1.0          0          3        2        2      2.0      2.5   

   score_5        clarity       coherent  \
0      1.0        average          worst   
1      1.0      excellent          worst   
2      1.5          worst  above_average   
3      1.0          worst          worst   
4      1.0  above_average          worst   

                                           EssayText  
0  Some additional information that we would need...  
1  After reading the expirement, I realized that ...  
2  What you need is more trials, a control set up...  
3  The student should list what rock is better an...

1657

In [104]:
p = get_vector(bert_embedding,s)

In [109]:
import pandas as pd
new_tfidf_features = pd.DataFrame(pp)

TypeError: 'numpy.float64' object is not iterable

In [106]:
new_tfidf_features

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.209471,0.004496,-0.115139,0.155171,0.477535,-0.195618,-0.013060,0.231690,0.323169,-0.301131,...,-0.406724,-0.040973,-0.027652,-0.111346,-0.153751,0.059796,0.271627,-0.166859,0.266631,0.047705
1,0.110659,-0.038571,0.083564,0.122763,0.229474,-0.293610,0.138813,0.316974,0.124054,-0.196293,...,-0.045047,0.011148,0.041355,-0.202223,-0.124132,-0.032951,0.210262,0.048532,-0.073355,0.153868
2,-0.105877,-0.126139,-0.004517,0.353253,0.354069,-0.232021,-0.035967,0.070528,-0.000716,-0.277295,...,-0.259656,-0.303123,-0.236008,-0.048549,-0.055279,0.216803,0.012138,-0.366128,0.102854,0.375488
3,-0.306760,-0.072570,-0.030354,-0.015995,0.274578,-0.018717,0.272740,0.152896,0.272880,-0.403173,...,0.159280,-0.087001,0.050957,0.070430,-0.106263,0.094872,0.043582,-0.020065,0.508369,0.091591
4,0.015524,0.107056,-0.131454,0.036079,0.357085,0.104878,0.202547,0.311471,0.347213,-0.294072,...,-0.486928,-0.445874,0.028652,-0.181518,-0.186059,0.195187,0.094222,-0.200974,0.422862,0.386694
5,0.086325,0.000576,0.049581,0.331226,0.195899,-0.161240,-0.333660,0.332043,-0.124765,0.001668,...,-0.218233,-0.162323,0.000013,-0.259459,-0.218776,-0.109985,0.111837,-0.036943,0.089155,-0.009786
6,0.087262,0.135113,0.010001,0.085915,0.297967,0.039862,0.267242,0.216932,0.238011,-0.373344,...,-0.372051,-0.061580,0.220201,-0.275639,-0.227072,0.027100,0.359274,-0.344252,0.250150,0.117292
7,0.125913,-0.294783,0.034882,0.252560,0.392762,-0.150249,0.012134,0.098217,0.112444,-0.382418,...,-0.113561,-0.075491,0.008705,-0.176691,-0.428926,-0.254321,0.388297,-0.185857,0.239657,-0.030906
8,-0.191039,-0.075475,-0.021523,0.143183,0.322996,-0.492606,-0.184364,0.484266,-0.272178,-0.287135,...,-0.116671,-0.097174,0.232923,-0.392272,-0.257715,-0.263478,-0.079174,0.080497,0.236805,-0.225707
9,0.114660,-0.129702,0.189297,-0.002228,0.332263,-0.325988,0.274241,0.147959,0.345936,-0.199805,...,-0.062633,0.156715,-0.182623,-0.198911,-0.137349,0.023985,0.108520,-0.312357,0.231118,0.127369
